In [2]:
from pathlib import Path
from pysota.core import Publication
from pysota.process import Persistence
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd
import spacy
from gensim.models import CoherenceModel

nlp = spacy.load('en_core_web_lg')

In [3]:
from bertopic import BERTopic

In [25]:
exclude = [
    "representation", 
    "learning", 
    "learn", 
    "training", 
    "train", 
    "supervision",
    "supervised",
    "supervise",
    "method", 
    "model", 
    "datum", 
    "self" ,
    "task", 
]

In [ ]:
def preprocess(text, exclude=exclude):
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha and token.lemma_ not in exclude and token.lemma_ != 'ADV'
    ]
    return ' '.join(tokens)

In [12]:
cluster = 0

In [27]:
db: list[Publication] = Persistence.load_files(path=Path('../results/clean'), query_name="")

In [28]:
ds = [preprocess(i.abstract) for i in db]

In [32]:
' '.join(ds[0])

'dimensional object move relative observer change occur observer image plane visual compute start idea good visual transform linearly scene motion theory group equivalent combination elementary irreducible derive strike relationship irreducibility statistical dependency structure show restricted condition irreducible decorrelate partial observability induce perspective projection scene image plane motion group linear action space image necessary perform inference latent transform linearly idea demonstrate rotate norb object employ latent non commutative rotation group'

In [29]:
ds

[['dimensional',
  'object',
  'move',
  'relative',
  'observer',
  'change',
  'occur',
  'observer',
  'image',
  'plane',
  'visual',
  'compute',
  'start',
  'idea',
  'good',
  'visual',
  'transform',
  'linearly',
  'scene',
  'motion',
  'theory',
  'group',
  'equivalent',
  'combination',
  'elementary',
  'irreducible',
  'derive',
  'strike',
  'relationship',
  'irreducibility',
  'statistical',
  'dependency',
  'structure',
  'show',
  'restricted',
  'condition',
  'irreducible',
  'decorrelate',
  'partial',
  'observability',
  'induce',
  'perspective',
  'projection',
  'scene',
  'image',
  'plane',
  'motion',
  'group',
  'linear',
  'action',
  'space',
  'image',
  'necessary',
  'perform',
  'inference',
  'latent',
  'transform',
  'linearly',
  'idea',
  'demonstrate',
  'rotate',
  'norb',
  'object',
  'employ',
  'latent',
  'non',
  'commutative',
  'rotation',
  'group'],
 ['thesis',
  'focus',
  'sequence',
  'time',
  'space',
  'aim',
  'improve',


In [19]:
model = BERTopic()

topics, probs = model.fit_transform(ds)

In [20]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11,-1_the_graph_and_of,"[the, graph, and, of, learning, to, node, in, ...",[Graph representation learning has become a ma...
1,0,73,0_the_of_and_to,"[the, of, and, to, learning, in, for, represen...",[ Advances in neural network based classifiers...
2,1,14,1_the_learning_in_that,"[the, learning, in, that, to, representation, ...","[ In contextual linear bandits, the reward fun..."


In [24]:
for topic in model.get_topic_info().Representation:
    print(topic)

['the', 'graph', 'and', 'of', 'learning', 'to', 'node', 'in', 'on', 'representations']
['the', 'of', 'and', 'to', 'learning', 'in', 'for', 'representation', 'we', 'on']
['the', 'learning', 'in', 'that', 'to', 'representation', 'of', 'we', 'representations', 'and']
